In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath('../'))
from sklearn.model_selection import train_test_split

from TumorDetection.data.loader import DataPathLoader
from TumorDetection.data.dataset import TorchDataset, Dataset
from TumorDetection.utils.dict_classes import DataPathDir, Device, ReportingPathDir
from TumorDetection.models.efsnet import EFSNet
from TumorDetection.models.utils.lightning_model import LightningModel
from TumorDetection.models.utils.trainer import Trainer

In [2]:
dp = DataPathLoader(DataPathDir.get('dir_path'))
paths = dp()
_, val_paths = train_test_split(paths, test_size=100, random_state=0, shuffle=True)
val_td = TorchDataset(val_paths)

model = LightningModel(EFSNet())
model = model.__class__.load(
    ckpt_dir = os.path.join(
        ReportingPathDir.get('dir_path'), 'ckpt'), 
    model_name = model.model_name,
    model = EFSNet,
    nn_kwargs = model.model.kwargs)

seg_pred, lab_pred = model.predict(val_td)

Found pretrained model: EFSNet.ckpt


TypeError: conv2d() received an invalid combination of arguments - got (tuple, Parameter, NoneType, tuple, tuple, tuple, int), but expected one of:
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, tuple of ints padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: ([31;1mtuple of (Tensor, Tensor, Tensor)[0m, [31;1mParameter[0m, [31;1mNoneType[0m, [31;1mtuple of (int, int)[0m, [31;1mtuple of (int, int)[0m, [31;1mtuple of (int, int)[0m, [32;1mint[0m)
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, str padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: ([31;1mtuple of (Tensor, Tensor, Tensor)[0m, [31;1mParameter[0m, [31;1mNoneType[0m, [31;1mtuple of (int, int)[0m, [31;1mtuple of (int, int)[0m, [31;1mtuple of (int, int)[0m, [32;1mint[0m)
